In [42]:
import os.path

import torch
import torch.nn as nn
import tqdm
from lightgbm import train
from torch.utils.data import DataLoader
from torchvision.transforms import transforms


class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.C1 = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4), # 55
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2), # 27
        )
        self.C2 = nn.Sequential(
            nn.Conv2d(96,256,kernel_size=5, padding=2,stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2), # 13
        )
        self.C3 = nn.Sequential(
            nn.Conv2d(256,384,kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
        )
        self.C4 = nn.Sequential(
            nn.Conv2d(384,384,kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
        )
        self.C5 = nn.Sequential(
            nn.Conv2d(384,256,kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2), 
        )
        self.FC6 = nn.Sequential(
            nn.Conv2d(256,4096,kernel_size=6),
            nn.ReLU(inplace=True),
            nn.Dropout(),
        )
        self.FC7 = nn.Sequential(
            nn.Conv2d(4096,4096,kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Dropout(),
        )
        self.FC8 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(4096,10),
        )
    def forward(self,x):
        '''
        :param x: x.shape = (batch_size, 3, 224, 224) 
        :return: 
        '''
        input_data = x
        x = self.C1(x)
        x = self.C2(x)
        x = self.C3(x)
        x = self.C4(x)
        x = self.C5(x)
        x = self.FC6(x)
        x = self.FC7(x)
        x = self.FC8(x)
        return x      

In [43]:
def ge(width,kernel,stride,padding=0):
    return (width + 2*padding - kernel) // stride + 1

In [44]:
from torchvision.datasets import CIFAR10
def LoadCIFAR10():
    train_dataset = CIFAR10('../dataset/',train=True,download=True,transform=transforms.Compose([
        transforms.Resize((227,227)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]))
    test_dataset = CIFAR10('../dataset/',train=False,download=True,transform=transforms.Compose([
        transforms.Resize((227,227)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]))
    return train_dataset,test_dataset

In [45]:
from torch.autograd import Variable
from torch.utils.data import DataLoader

import torch.nn as nn

class Trainer(object):
    # 初始化模型、配置参数、优化器和损失函数
    def __init__(self, model, config):
        self._model = model
        self._config = config
        self._optimizer = torch.optim.Adam(self._model.parameters(),\
                                           lr=config['lr'], weight_decay=config['l2_regularization'])
        self.loss_func = nn.CrossEntropyLoss()
    # 对单个小批量数据进行训练，包括前向传播、计算损失、反向传播和更新模型参数
    def _train_single_batch(self, images, labels):
        y_predict = self._model(images)

        loss = self.loss_func(y_predict, labels)
        # 先将梯度清零,如果不清零，那么这个梯度就和上一个mini-batch有关
        self._optimizer.zero_grad()
        # 反向传播计算梯度
        loss.backward()
        # 梯度下降等优化器 更新参数
        self._optimizer.step()
        # 将loss的值提取成python的float类型
        loss = loss.item()

        # 计算训练精确度
        # 这里的y_predict是一个多个分类输出，将dim指定为1，即返回每一个分类输出最大的值以及下标
        _, predicted = torch.max(y_predict.data, dim=1)
        return loss, predicted

    def _train_an_epoch(self, train_loader, epoch_id):
        """
        训练一个Epoch，即将训练集中的所有样本全部都过一遍
        """
        # 设置模型为训练模式，启用dropout以及batch normalization
        self._model.train()
        total = 0
        correct = 0
        # 从DataLoader中获取小批量的id以及数据
        for images, labels in tqdm.tqdm(train_loader):
            images = Variable(images)
            labels = Variable(labels)
            if self._config['use_cuda'] is True:
                images, labels = images.cuda(), labels.cuda()

            loss, predicted = self._train_single_batch(images, labels)

            # 计算训练精确度
            total += labels.size(0)
            correct += (predicted == labels.data).sum()

            # print('[Training Epoch: {}] Batch: {}, Loss: {}'.format(epoch_id, batch_id, loss))
        print('Training Epoch: {}, accuracy rate: {}%%'.format(epoch_id, correct / total * 100.0))

    def train(self, train_dataset):
        # 是否使用GPU加速
        self.use_cuda()
        for epoch in range(self._config['num_epoch']):
            print('-' * 20 + ' Epoch {} starts '.format(epoch) + '-' * 20)
            # 构造DataLoader
            data_loader = DataLoader(dataset=train_dataset, batch_size=self._config['batch_size'], shuffle=True)
            # 训练一个轮次
            self._train_an_epoch(data_loader, epoch_id=epoch)

    # 用于将模型和数据迁移到GPU上进行计算，如果CUDA不可用则会抛出异常
    def use_cuda(self):
        if self._config['use_cuda'] is True:
            assert torch.cuda.is_available(), 'CUDA is not available'
            torch.cuda.set_device(self._config['device_id'])
            self._model.cuda()

    # 保存训练好的模型
    def save(self):
        self._model.saveModel()
def Construct_DataLoader(dataset, batchsize,shuffle=True):
    return DataLoader(dataset=dataset, batch_size=batchsize, shuffle=shuffle)

In [46]:
alexnet_config = \
{
    'num_epoch': 20,              # 训练轮次数
    'batch_size': 200,            # 每个小批量训练的样本数量
    'lr': 1e-3,                   # 学习率
    'l2_regularization':1e-4,     # L2正则化系数
    'device_id': 0,               # 使用的GPU设备的ID号
    'use_cuda': True,             # 是否使用CUDA加速
    'model_name': './AlexNet.model' # 保存模型的文件名
}

train_dataset, test_dataset = LoadCIFAR10()
alexNet = AlexNet()
trainer = Trainer(model=alexNet, config=alexnet_config)
trainer.train(train_dataset)
trainer.save()

alexNet.eval()
alexNet.cuda()
correct = 0
total = 0
# 对测试集中的每个样本进行预测，并计算出预测的精度
for images, labels in Construct_DataLoader(test_dataset, alexnet_config['batch_size'],False):
    images = Variable(images)
    labels = Variable(labels)
    if alexnet_config['use_cuda']:
        images = images.cuda()
        labels = labels.cuda()

    y_pred = alexNet(images)
    _, predicted = torch.max(y_pred.data, 1)
    total += labels.size(0)
    temp = (predicted == labels.data).sum()
    correct += temp
print('Accuracy of the model on the test images: %.2f%%' % (100.0 * correct / total))

Files already downloaded and verified
Files already downloaded and verified
-------------------- Epoch 0 starts --------------------


  1%|          | 3/250 [00:09<12:36,  3.06s/it]


KeyboardInterrupt: 